In [1]:
import requests
import numpy as np
import pandas as pd
import pymysql as mysql
from pprint import pprint
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from termcolor import colored
from sqlalchemy import create_engine, MetaData
from sqlalchemy_schemadisplay import create_schema_graph
from IPython.display import Image
import base64
warnings.filterwarnings('ignore')

Alternative APIs (rapid API), username trevsauer2023

- Crime Data By ZipCode:
https://rapidapi.com/zyla-labs-zyla-labs-default/api/crime-data-by-zipcode-api/

- EveryEarthquake: 
https://rapidapi.com/dbarkman/api/everyearthquake/



In [48]:
city = "San Diego"

# Covid API

In [49]:
# make the API call
# link to documentation: https://rapidapi.com/axisbits-axisbits-default/api/covid-19-statistics/
url = "https://covid-19-statistics.p.rapidapi.com/reports"

# optional parameter selected to querying the API
querystring = {"city_name":city}

# API credentials
headers = {
	"X-RapidAPI-Key": "ad2cc6becemsh2e08af212b398c8p15d59djsn310073516954",
	"X-RapidAPI-Host": "covid-19-statistics.p.rapidapi.com"
}

# get raw response query from the API
response = requests.request("GET", url, headers=headers, params=querystring)
print(response.text)

{"data":[{"date":"2023-02-07","confirmed":12004743,"deaths":100033,"recovered":0,"confirmed_diff":-6370,"deaths_diff":-315,"recovered_diff":0,"last_update":"2023-02-08 04:20:59","active":11904710,"active_diff":-6055,"fatality_rate":0.0083,"region":{"iso":"USA","name":"US","province":"California","lat":"36.1162","long":"-119.6816","cities":[{"name":"San Diego","date":"2023-02-07","fips":6073,"lat":"33.03484597","long":"-116.7365326","confirmed":1057398,"deaths":5728,"confirmed_diff":0,"deaths_diff":0,"last_update":"2023-02-08 04:20:59"}]}}]}


In [50]:
# Convert the response to a dictionary
data = response.json()

# Create a Pandas dataframe from the dictionary
df = pd.DataFrame(data)

# Unpack the dictionaries in the column into separate columns
df_unpacked = pd.json_normalize(df['data'])

# merge the new columns back into the original dataframe
df_1 = df.merge(df_unpacked, left_index=True, right_index=True)

# drop the original column containing the dictionaries
df_2 = df_1.drop('data', axis=1)

# extract the dictionary from region.cities column for flattening
col_dic = df_2['region.cities'].iloc[0]

# turn the extracted dictionary into its on dataframe 
flatten_col = pd.DataFrame(col_dic)

# merge the two dataframes
result = df_2.merge(flatten_col, left_index=True, right_index=True)

# show the values of all the columns
row_values = result.iloc[0].to_dict()
row_values

{'date_x': '2023-02-07',
 'confirmed_x': 12004743,
 'deaths_x': 100033,
 'recovered': 0,
 'confirmed_diff_x': -6370,
 'deaths_diff_x': -315,
 'recovered_diff': 0,
 'last_update_x': '2023-02-08 04:20:59',
 'active': 11904710,
 'active_diff': -6055,
 'fatality_rate': 0.0083,
 'region.iso': 'USA',
 'region.name': 'US',
 'region.province': 'California',
 'region.lat': '36.1162',
 'region.long': '-119.6816',
 'region.cities': [{'name': 'San Diego',
   'date': '2023-02-07',
   'fips': 6073,
   'lat': '33.03484597',
   'long': '-116.7365326',
   'confirmed': 1057398,
   'deaths': 5728,
   'confirmed_diff': 0,
   'deaths_diff': 0,
   'last_update': '2023-02-08 04:20:59'}],
 'name': 'San Diego',
 'date_y': '2023-02-07',
 'fips': 6073,
 'lat': '33.03484597',
 'long': '-116.7365326',
 'confirmed_y': 1057398,
 'deaths_y': 5728,
 'confirmed_diff_y': 0,
 'deaths_diff_y': 0,
 'last_update_y': '2023-02-08 04:20:59'}

# Weather API(s)

In [54]:
url = "https://forecast9.p.rapidapi.com/rapidapi/forecast/%s/hourly/" % city

headers = {
	"X-RapidAPI-Key": "ad2cc6becemsh2e08af212b398c8p15d59djsn310073516954",
	"X-RapidAPI-Host": "forecast9.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

'{"location":{"code":"CO3669947","name":"San Diego","timezone":"America\\/Bogota","coordinates":{"latitude":10.3362,"longitude":-73.182}},"forecast":{"items":[{"date":"2023-02-08T06:00:00Z","period":3,"freshSnow":0,"weather":{"state":60,"text":"leichter Regen","icon":"n_60.svg"},"sunHours":null,"rainHours":null,"prec":{"sum":0.12,"probability":10,"sumAsRain":null,"class":1},"temperature":{"avg":24},"pressure":972,"relativeHumidity":63,"clouds":{"high":8,"low":0,"middle":10},"wind":{"unit":"km\\/h","direction":"Nordostwind","text":"NO","avg":16.93,"min":null,"max":null,"gusts":{"value":null,"text":""},"significationWind":false},"windchill":{"avg":25,"min":null,"max":null},"snowLine":{"avg":null,"min":null,"max":null,"unit":"m"},"isNight":true},{"date":"2023-02-08T09:00:00Z","period":3,"freshSnow":0,"weather":{"state":0,"text":"klar","icon":"n_0.svg"},"sunHours":null,"rainHours":null,"prec":{"sum":0,"probability":0,"sumAsRain":null,"class":0},"temperature":{"avg":24},"pressure":972,"rela

In [57]:
# convert the response to a dictionary
data = response.json()

# # create a Pandas dataframe from the dictionary
df = pd.DataFrame(data)

# # Unpack the dictionaries in the column into separate columns
df_unpacked = pd.json_normalize(df['data'])

# # Merge the new columns back into the original dataframe
# df_1 = df.merge(df_unpacked, left_index=True, right_index=True)

KeyError: 'data'

In [13]:
url = "https://crime-data-by-zipcode-api.p.rapidapi.com/crime_data"

querystring = {"zip":"94109"}

headers = {
	"X-RapidAPI-Key": "ad2cc6becemsh2e08af212b398c8p15d59djsn310073516954",
	"X-RapidAPI-Host": "crime-data-by-zipcode-api.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

df = pd.DataFrame(response)

print(response.text)


{"Overall":{"Zipcode":"94109","Overall Crime Grade":"F","Violent Crime Grade":"F","Property Crime Grade":"F","Other Crime Grade":"F","Fact":"A crime occurs every 1 hour 11 minutes (on average) in 94109.","Risk":"300%","Risk Detail":"Your home is 300% more likely to be robbed with no home security system."},"Crime BreakDown":[{"Violent Crime Rates":{"Assault":"11.35","Robbery":"4.370","Rape":"0.4715","Murder":"0.0509"},"0":{"Total Violent Crime":"16.24","Total Violent Crime Score":"(F)"}},{"Property Crime Rates":{"Theft":"59.22","Vehicle Theft":"3.198","Burglary":"12.50","Arson":"3.770"},"0":{"Total Property Crime":"78.69","Total Property Crime Score":"(F)"}},{"Other Crime Rates":{"Kidnapping":"0.4780","Drug Crimes":"25.45","Vandalism":"7.619","Identity Theft":"1.047","Animal Cruelty":"0.0570"},"0":{"Total Other Rate":"34.65","Total Other Score":"(F)"}}],"Crime Rates Nearby":[{"Nearby Zip":"94108, CA","Overall Crime Grade":"F","Violent Crime Grade":"F","Property Crime Grade":"F"},{"Near

In [3]:
api_key = "aV2EeC5eASWbRJVEBK6BEJr9a9pntz40"
secret = "eZtBWBOugMrfxVjo"
oauth_endpoint = "https://api.precisely.com/oauth/token"
token = base64.b64encode(f"{api_key}:{secret}".encode("utf-8")).decode("utf-8")
print(token)

headers = {
    "Authorization": f"Basic {token}",
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(oauth_endpoint, headers=headers, data={"grant_type": "client_credentials"})
response.raise_for_status()
access_token = response.json()["access_token"]

YVYyRWVDNWVBU1diUkpWRUJLNkJFSnI5YTlwbnR6NDA6ZVp0QldCT3VnTXJmeFZqbw==


In [4]:
# Set the API endpoint URL

# url = "https://api.precisely.com/risks/v1/crime/byaddress"

url = "https://api.precisely.com/typeahead/v1/locations?latitude=51.1789&longitude=1.8262&searchText=StoneHenge"
# url = "https://api.precisely.com/typeahead/v1/locations?latitude=51.1789N&longitude=1.8262W&searchText="

# Set the headers for the API request
headers = {

    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Make a GET request to the API
response = requests.get(url, headers=headers)

# Check the status code of the response
if response.status_code == 200:

    # If the response is successful, extract the data
    data = response.json()

    # Convert the data to a Pandas dataframe
    df = pd.DataFrame(data)

    # Print the dataframe
    print(df)
    
else:
    # If the response is unsuccessful, print the error message
    print(f"Failed to retrieve data from Precisely API. Error: {response.text}")

Empty DataFrame
Columns: [location]
Index: []


In [37]:
print(data)

{'location': []}


This code makes a GET request to the Precisely API using the requests.get() method and passes in the URL and headers as arguments. The response from the API is then stored in the response variable. You can check the status code of the response to see if the request was successful (status code 200) or not, and retrieve the data from the response using the response.json() method.

Note that you need to replace "your_api_key" with your actual Precisely API key.

When incorporating the N/S and E/W indicators into latitude and longitude values for API calls, the standard way is to use a positive number for the Northern Hemisphere and Eastern Hemisphere, and a negative number for the Southern Hemisphere and Western Hemisphere.


In [29]:
35999 / 3600

9.999722222222223